In [1]:
import keras

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.datasets import reuters

In [4]:
(train_data,train_labels),(test_data,test_labels)=reuters.load_data(num_words=10000)

2113536/2110848 [==============================] - 8s 4us/step


In [5]:
train_data[10]

[1,
 245,
 273,
 207,
 156,
 53,
 74,
 160,
 26,
 14,
 46,
 296,
 26,
 39,
 74,
 2979,
 3554,
 14,
 46,
 4689,
 4329,
 86,
 61,
 3499,
 4795,
 14,
 61,
 451,
 4329,
 17,
 12]

In [7]:
word_index=reuters.get_word_index()
reverse_word_index=dict([(v,k) for (k,v)in word_index.items()])
decoded_newswire=' '.join([reverse_word_index.get(i-3,'?')for i in train_data[0]])

557056/550378 [==============================] - 2s 3us/step


In [8]:
import numpy as np

def vs(s,dim=10000):
    results=np.zeros((len(s),dim))
    for i,s in enumerate(s):
        results[i,s]=1.
    return results

In [9]:
x_train=vs(train_data)
x_test=vs(test_data)

In [11]:
def to_one_hot(labels,dim=46):
    results=np.zeros((len(labels),dim))
    for i, labels in enumerate(labels):
        results[i,label]=1.
    return results

In [12]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels=to_categorical(train_labels)
one_hot_test_labels=to_categorical(test_labels)


In [13]:
from keras import models
from keras import layers


In [20]:
model=models.Sequential()
model.add(layers.Dense(64,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))

In [21]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [23]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 2s 197us/step - loss: 2.4794 - acc: 0.4961 - val_loss: 1.6305 - val_acc: 0.6260
Epoch 2/20
7982/7982 [==============================] - 1s 158us/step - loss: 1.3895 - acc: 0.6909 - val_loss: 1.3094 - val_acc: 0.6990
Epoch 3/20
7982/7982 [==============================] - 1s 158us/step - loss: 1.0571 - acc: 0.7640 - val_loss: 1.1371 - val_acc: 0.7490
Epoch 4/20
7982/7982 [==============================] - 1s 158us/step - loss: 0.8243 - acc: 0.8213 - val_loss: 1.0202 - val_acc: 0.7870
Epoch 5/20
7982/7982 [==============================] - 1s 159us/step - loss: 0.6551 - acc: 0.8562 - val_loss: 0.9649 - val_acc: 0.7940
Epoch 6/20
7982/7982 [==============================] - 1s 167us/step - loss: 0.5088 - acc: 0.8898 - val_loss: 0.9668 - val_acc: 0.7930
Epoch 7/20
7982/7982 [==============================] - 1s 161us/step - loss: 0.4039 - acc: 0.9146 - val_loss: 1.0477 - val_acc: 0.7820


Our approach reaches an accuracy of ~78%. With a balanced binary classification problem, the accuracy reached by a purely random classifier would be 50%, but in our case it is closer to 19%, so our results seem pretty good, at least when compared to a random baseline:

In [18]:
predictions=model.predict(x_test)

In [19]:
np.sum(predictions[0])

1.0